In [1]:
#Librerías
import ee #earthengine
import geemap #Alternativa a GEE paquete
import matplotlib.pyplot as plt #Generación de gráficos
import pandas as pd
import numpy as np
import os

In [2]:
ee.Initialize()

In [3]:
Map = geemap.Map()

In [4]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [5]:
AOI = 'D:/DOCS/PATIA/Sph.AltoPatia/AltoPatia.shp'

In [6]:
AOI = geemap.shp_to_ee(AOI)

In [7]:
#Map.addLayer(AOI, {}, 'AOI')

# **Vegetation Health Index (VHI)**

<p>The <a href="https://un-spider.org/advisory-support/recommended-practices/recommended-practice-agriculture-drought-monitoring/step-by-step">VHI</a> is a combination of the constructed VCI and TCI and can be used effectively for drought assessments. It can be calculated using the following formula. </p><br>

$VHI = \alpha * VCI + (1 - \alpha) * TCI$

<p>where $\alpha$ is the weight to measure the contribution of the $VCI$ and $TCI$ for assessing the status of drought. Generally, $\alpha$ is set as 0.5 because it is difficult to distinguish the contribution of the surface temperature and the $NDVI$ when measuring drought stress. </p><br>

# **<p style="background-color:Tomato;">INGRESO DE DATOS</p>**

In [8]:
fechaI = "2001-01-01" # yyyy-mm-dd --> Ingresar fecha inicial
fechaF = "2021-12-31" # yyyy-mm-dd --> Ingresar fecha final

In [9]:
Anios = ee.List.sequence(2001, 2021) # A partir de la fecha registrada en la celda anterior 
                              # incluir el rango de años correspondiente --> Año inicial, Año final + 1
Anios

In [10]:
Meses = ee.List.sequence(1,12)
Meses

In [11]:
MOD11A2_006 = ee.ImageCollection('MODIS/006/MOD11A2').filterBounds(AOI).filterDate(fechaI,fechaF) # Para TCI
MOD13Q1_006 = ee.ImageCollection('MODIS/006/MOD13Q1').filterBounds(AOI).filterDate(fechaI,fechaF) # Para VCI

# **<p style="background-color:Tomato;">PROCESAMIENTO DE DATOS</p>**

In [12]:
# Valor mínimo y máximo de temperaturas superficiales terrestres
minLST = MOD11A2_006.select('LST_Day_1km').min()
maxLST = MOD11A2_006.select('LST_Day_1km').max()

# Valor mínimo y máximo de NDVIs
minNDVI = MOD13Q1_006.select('NDVI').min()
maxNDVI = MOD13Q1_006.select('NDVI').max()

In [13]:
Lw = ee.List([])
for i in range(2001,2022):
    for j in range(1,13):
        w = MOD11A2_006.select('LST_Day_1km').filter(ee.Filter.calendarRange(ee.Number(i), ee.Number(i), 'year')).filter(ee.Filter.calendarRange(ee.Number(j), ee.Number(j), 'month')).mean()
        w = w.set('year', i)
        w = w.set('month', j)
        w = w.set('system:time_start', ee.Date.fromYMD(i, j, 1))
        Lw = Lw.add(ee.Image(w))
W = ee.ImageCollection.fromImages(Lw)

In [14]:
Nw = ee.List([])
for i in range(2001,2022):
    for j in range(1,13):
        w = MOD13Q1_006.select('NDVI').filter(ee.Filter.calendarRange(ee.Number(i), ee.Number(i), 'year')).filter(ee.Filter.calendarRange(ee.Number(j), ee.Number(j), 'month')).mean()
        w = w.set('year', i)
        w = w.set('month', j)
        w = w.set('system:time_start', ee.Date.fromYMD(i, j, 1))
        Nw = Nw.add(ee.Image(w))
NDVIj = ee.ImageCollection.fromImages(Nw)

In [15]:
def TCIf(imgCol):
    TCI = (imgCol.subtract(minLST)).divide(maxLST.subtract(minLST)).clip(AOI)
    return TCI

In [16]:
def VCIf(imgCol):
    VCI = (imgCol.subtract(minNDVI)).divide(maxNDVI.subtract(minNDVI)).clip(AOI)
    return VCI

In [17]:
TCI = W.map(TCIf)

In [18]:
VCI = NDVIj.map(VCIf)

In [19]:
def AlphaVCIf(imgC1):
    AlphaVCI = (imgC1.multiply(0.5))
    return AlphaVCI

In [20]:
def AlphaTCIf(imgC2):
    AlphaTCI = (imgC2.multiply(0.5))
    return AlphaTCI

In [21]:
AlphaVCI = VCI.map(AlphaVCIf)

In [22]:
AlphaTCI = TCI.map(AlphaTCIf)

In [23]:
Vw = ee.List([])

Alpha_VCI = AlphaVCI.toList(AlphaTCI.size())
Alpha_TCI = AlphaTCI.toList(AlphaTCI.size())
for i in range(252): # verificar con .getInfo() el valor de 'system:index': '###', para este caso son 251
    v = ee.Image(Alpha_VCI.get(i)).add(ee.Image(Alpha_TCI.get(i)))
    Vw = Vw.add(ee.Image(v))
VHI = ee.ImageCollection.fromImages(Vw)

In [24]:
vis =  ['d7191c', 'fdae61', 'ffffc0', 'a6d96a', '1a9641']
visParams = {'min': -1, 'max': 1, 'palette': vis}

In [25]:
#Map.addLayer(TCI, visParams, 'TCI')

In [26]:
#Map.addLayer(VCI, visParams, 'VCI')

In [27]:
#Map.addLayer(VHI, visParams, 'VHI')

In [28]:
#Criterios de clasificacion TCI
def ClasificarImgTCI(ImgCol):
    image02 = ee.Image(ImgCol.lt(0.1).And(ImgCol.gte(-1)))
    image04 = ee.Image(((ImgCol.gte(0.1)).And(ImgCol.lt(0.2))).multiply(2))
    image06 = ((ImgCol.gte(0.2)).And(ImgCol.lt(0.3))).multiply(3)
    image08 = ((ImgCol.gte(0.3)).And(ImgCol.lt(0.4))).multiply(4)
    image10 = (ImgCol.gte(0.4)).multiply(5)
    Drought_Index = (image02.add(image04).add(image06).add(image08).add(image10))
    Drought_Index = Drought_Index.float()
    return Drought_Index

In [29]:
#Criterios de clasificacion VCI
def ClasificarImgVCI(ImgCol):
    image02 = ee.Image(ImgCol.lt(0.1).And(ImgCol.gte(-1)))
    image04 = ee.Image(((ImgCol.gte(0.1)).And(ImgCol.lt(0.2))).multiply(2))
    image06 = ((ImgCol.gte(0.2)).And(ImgCol.lt(0.3))).multiply(3)
    image08 = ((ImgCol.gte(0.3)).And(ImgCol.lt(0.4))).multiply(4)
    image10 = (ImgCol.gte(0.4)).multiply(5)
    Drought_Index = (image02.add(image04).add(image06).add(image08).add(image10))
    Drought_Index = Drought_Index.float()
    return Drought_Index

In [30]:
#Criterios de clasificacion VHI

def ClasificarImgVHI(ImgCol):
    image02 = ee.Image(ImgCol.lt(0.1).And(ImgCol.gte(-1)))
    image04 = ee.Image(((ImgCol.gte(0.1)).And(ImgCol.lt(0.2))).multiply(2))
    image06 = ((ImgCol.gte(0.2)).And(ImgCol.lt(0.3))).multiply(3)
    image08 = ((ImgCol.gte(0.3)).And(ImgCol.lt(0.4))).multiply(4)
    image10 = (ImgCol.gte(0.4)).multiply(5)
    Drought_Index = (image02.add(image04).add(image06).add(image08).add(image10))
    Drought_Index = Drought_Index.float()
    return Drought_Index

In [31]:
IndiceSequiaTCI = VCI.map(ClasificarImgTCI)

In [32]:
IndiceSequiaVCI = VCI.map(ClasificarImgVCI)

In [33]:
IndiceSequiaVHI = VHI.map(ClasificarImgVHI)

In [34]:
vis2 =  ['d7191c', 'fdae61', 'ffffc0', 'a6d96a', '1a9641'] #Extreme (rojo),Severe (anaranjado), Moderate (amarillo), Mild (verde claro) and No Drought (verde oscuro)
visParams2 = {'min': 1, 'max': 5, 'palette': vis2}

In [35]:
#Map.addLayer(IndiceSequiaTCI, visParams2 , 'IndiceSequia TCI')

In [36]:
#Map.addLayer(IndiceSequiaVCI, visParams2 , 'IndiceSequia VCI')

In [37]:
#Map.addLayer(IndiceSequiaVHI, visParams2 , 'IndiceSequia VHI')#

In [38]:
#Map.set_plot_options(add_marker_cluster= True)

In [39]:
# Indices de sequía

# 'Extreme' = 1
# 'Severe' = 2
# 'Moderate' = 3
# 'Mild' = 4
# 'No Drought' = 5

In [40]:
Geometria = ee.Geometry.Rectangle([-77.9612271805933119,0.6744206529957637, -76.5544471458635485,2.5194416258749825])

## PCA

PCA transform is a rotational transform in which the new basis is orthonormal, but the axes are determined from statistics of the input image, rather than empirical data.  Specifically, the new basis is the eigenvectors of the image's variance-covariance matrix.  As a result, the PCs are uncorrelated.  To demonstrate, use the Landsat 8 image, converted to an array image:

In [41]:
VHIBands = VHI.select("NDVI").toBands()
VHIBands = VHIBands.toArray()
VHIBands.getInfo()

{'type': 'Image',
 'bands': [{'id': 'array',
   'data_type': {'type': 'PixelType', 'precision': 'double', 'dimensions': 1},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [42]:
covar = VHIBands.reduceRegion(**{'reducer': ee.Reducer.covariance(),'geometry':Geometria,'maxPixels': 1e24, 'scale': 1000, 'crs': VHI.first().projection()})

In [43]:
covarArray = ee.Array(covar.get('array'))

In [44]:
eigens = covarArray.eigen()

In [45]:
eigenVectors = eigens.slice(1, 1)

In [46]:
principalComponents = ee.Image(eigenVectors).matrixMultiply(VHIBands.toArray(1))

In [47]:
pcImage = principalComponents.arrayProject([0]).arrayFlatten([['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7', 'pc8']])

In [48]:
Map.addLayer(pcImage.select('pc1'),{} , 'PCA')

# **<p style="background-color:Tomato;">Exportar primera imagén de cada modelo</p>**

In [49]:
#VHI_expFirst = geemap.ee_export_image(IndiceSequiaVHI.first(), 'D:/DOCS/PATIA/IMGENES/VHI.tif',scale=1000, region=Geometria)

In [50]:
#VCI = geemap.ee_export_image(IndiceSequiaVCI.first(), 'D:/DOCS/PATIA/IMGENES/VCI.tif',scale=250, region=Geometria)

In [51]:
#TCI_expFirst = geemap.ee_export_image(IndiceSequiaVHI.first(), 'D:/DOCS/PATIA/IMGENES/TCI.tif',scale=1000, region=Geometria)

# **<p style="background-color:Tomato;">Exportar colección de imagenes de cada modelo</p>**

In [52]:
#VHI = geemap.ee_export_image_collection(IndiceSequiaVHI, 'D:/DOCS/PATIA/IMGENES/VHI',scale=1000, region=Geometria)

In [53]:
#VCI = geemap.ee_export_image_collection(IndiceSequiaVCI, 'D:/DOCS/PATIA/IMGENES/VCI',scale=1000, region=Geometria)

In [54]:
#TCI = geemap.ee_export_image_collection(IndiceSequiaTCI, 'D:/DOCS/PATIA/IMGENES/TCI',scale=1000, region=Geometria)

In [55]:
#VHI_Raw = geemap.ee_export_image_collection(ee.ImageCollection(Vw), 'D:/DOCS/PATIA/IMGENES/VHI_Raw',scale=1000, region=Geometria)

In [56]:
#VCI_Raw = geemap.ee_export_image_collection(ee.ImageCollection(Nw), 'D:/DOCS/PATIA/IMGENES/VCI_Raw',scale=1000, region=Geometria)

In [57]:
#TCI_Raw = geemap.ee_export_image_collection(ee.ImageCollection(Lw), 'D:/DOCS/PATIA/IMGENES/TCI_Raw',scale=1000, region=Geometria)